In [1]:
import numpy as np

# Underflow and Overflow

One form of rounding error that is particularly devastating is *underflow*. Underflow occurs when numbers near zero are rounded to zero. Many functions behave qualitatively differently when their argument is zero rather than a small positive
number.
 
For example, we usually want to avoid division by zero (some software environments will raise exceptions when this occurs, others will return a result with a placeholder not-a-number value) or taking the logarithm of zero (this is
usually treated as $−∞$, which then becomes not-a-number if it is used for many further arithmetic operations).

In [2]:
1 / 0

ZeroDivisionError: integer division or modulo by zero

Another highly damaging form of numerical error is *overflow*. Overflow occurs when numbers with large magnitude are approximated as $∞$ or $−∞$. Further arithmetic will usually change these infinite values into not-a-number values.

One example of a function that must be stabilized against underflow and overflow is the softmax function. The softmax function is often used to predict the probabilities associated with a multinoulli distribution. The softmax function is defined to be

$$softmax(x)_{i} = \frac{exp(x_i)}{\sum_{j=1}^{n}{exp(x_j)}}$$

In [14]:
def softmax(x, i):
    return np.exp(x[i]) / np.sum(np.exp(x))

Consider what happens when all of the $x_i$ are equal to some constant $c$. Analytically, we can see that all of the outputs should be equal to $\frac{1}{n}$.

In [15]:
x = np.array([2, 2, 2, 2])
print softmax(x, 0)
print softmax(x, 1)
print softmax(x, 3)

0.25
0.25
0.25


If $c$ is very negative, then $exp(c)$ will underflow.

In [5]:
print "    -1: ", np.exp(-1)
print "   -10:", np.exp(-10)
print "  -100:", np.exp(-100)
print " -1000:", np.exp(-1000)
print "-10000:", np.exp(-10000)

    -1:  0.367879441171
   -10: 4.53999297625e-05
  -100: 3.72007597602e-44
 -1000: 0.0
-10000: 0.0


This means the denominator of the softmax will become 0, so the final result is undefined.

In [6]:
x = np.array([-1000] * 4)
print softmax(x, 0)
print softmax(x, 1)
print softmax(x, 3)

nan
nan
nan


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


When $c$ is very large and positive, $exp(c)$ will overflow

In [7]:
print "    1: ", np.exp(1)
print "   10:", np.exp(10)
print "  100:", np.exp(100)
print " 1000:", np.exp(1000)
print "10000:", np.exp(10000)

    1:  2.71828182846
   10: 22026.4657948
  100: 2.68811714182e+43
 1000: inf
10000: inf


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: overflow encountered in exp
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: overflow encountered in exp


again resulting in the expression as a whole being undefined

In [9]:
x = np.array([1000] * 4)
print softmax(x, 0)
print softmax(x, 1)
print softmax(x, 3)

nan
nan
nan


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


Both of these difficulties can be resolved by instead evaluating $softmax(z)$ where $z = x − max_{i} x_i$

In [17]:
def softmax(x, i):
    # version 2.0
    # avoiding overflow and underflow
    z = x - np.max(x)
    return np.exp(z[i]) / np.sum(np.exp(z))
x = np.array([2, 2, 2, 2])
print softmax(x, 0)
print softmax(x, 1)
print softmax(x, 3)

0.25
0.25
0.25


When elements in x are very small

In [19]:
x = np.array([-1000] * 4)
print softmax(x, 0)
print softmax(x, 1)
print softmax(x, 3)

0.25
0.25
0.25


in case they are very large

In [22]:
x = np.array([1000] * 4)
print softmax(x, 0)
print softmax(x, 1)
print softmax(x, 3)

0.25
0.25
0.25


There is still one small problem. *Underflow* in the numerator can still cause the expression as a whole to evaluate to zero. This means that if we implement $log softmax(x)$ by first running the softmax subroutine then passing the result to the log function, we could erroneously obtain $−∞$.

# Poor Conditioning

Consider the function $f(x) = A^{−1}x$. When $A ∈ R^{n×n}$ has an eigenvalue decomposition, its *condition number* is

$$max_{i,j} = \left | \frac{\lambda_i}{\lambda_j} \right | $$

This is the ratio of the magnitude of the largest and smallest eigenvalue. When this number is large, matrix inversion is particularly sensitive to error in the input.

# Gradient-Based Optimization